In [1]:
import pandas as pd #for handling csv and csv contents
from rdflib import Graph, Literal, RDF, URIRef, Namespace #basic RDF handling
from rdflib.namespace import FOAF , XSD, SSN, SOSA #most common namespaces
import urllib.parse #for parsing strings to URI's
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import stardog

In [2]:
df = pd.read_csv('Jasmin.csv')

In [3]:
df['Altitude']= pd.to_numeric(df['Altitude'],errors='coerce')
df['Altitude'].mean().round(6)

143.757143

In [4]:
df.head(10)

,ID,LocalDate,LocalTime,GMTDate,GMTTime,lat,long,Temperature,Speed,Direction,Altitude,Cov,HDOP,Distance,Count
0,1SAT32,2011-10-26,07:40:35 AM,2011-10-25,11:40:35 PM,5.437206,118.111301,24.5,0.24,0,NaN,0,1.64,0,1
1,2SAT32,2011-10-26,03:41:35 PM,2011-10-26,07:41:35 AM,5.433402,118.108162,29.0,0.76,0,NaN,0,1.74,548,2
2,3SAT32,2011-10-26,11:40:35 PM,2011-10-26,03:40:35 PM,5.433734,118.108599,27.0,0.86,0,NaN,0,2.18,61,3
3,4SAT32,2011-10-27,11:40:34 PM,2011-10-27,03:40:34 PM,5.455738,118.183640,28.0,0.16,0,NaN,0,1.89,8669,4
4,5SAT32,2011-10-28,07:42:53 AM,2011-10-27,11:42:53 PM,5.456503,118.187684,27.5,1.46,0,NaN,1,2.50,456,5
5,6SAT32,2011-10-28,03:41:05 PM,2011-10-28,07:41:05 AM,5.483344,118.232541,33.5,0.06,0,NaN,5,1.45,5800,6
6,7SAT32,2011-10-28,03:41:05 PM,2011-10-28,07:41:05 AM,5.483344,118.232541,33.5,0.06,0,NaN,5,1.45,0,7
7,8SAT32,2011-10-28,03:41:05 PM,2011-10-28,07:41:05 AM,5.483344,118.232541,33.5,0.06,0,NaN,5,1.45,0,8
8,9SAT32,2011-10-28,07:40:34 PM,2011-10-28,11:40:34 AM,5.479701,118.243606,29.0,0.14,0,NaN,4,0.95,1291,9
9,10SAT32,2011-10-28,07:40:34 PM,2011-10-28,11:40:34 AM,5.479701,118.243606,29.0,0.14,0,NaN,5,0.95,0,10


In [5]:
df.describe()

,lat,long,Temperature,Speed,Direction,Altitude,Cov,HDOP,Distance,Count
count,899.000000,899.000000,899.000000,899.000000,899.000000,7.000000,899.000000,899.000000,899.000000,899.000000
mean,5.509015,118.237317,28.888765,0.587164,1.329255,143.757143,2.497219,2.550044,640.407119,450.000000
std,0.073845,0.105518,4.016236,0.673300,17.429289,88.443143,2.170552,3.812480,917.534269,259.663243
min,5.392483,117.981100,-35.500000,0.000000,0.000000,101.500000,0.000000,0.710000,0.000000,1.000000
25%,5.465016,118.199810,28.000000,0.180000,0.000000,103.050000,0.000000,1.295000,145.500000,225.500000
50%,5.490733,118.248029,28.500000,0.370000,0.000000,114.900000,2.000000,1.830000,341.000000,450.000000
75%,5.546202,118.311595,30.000000,0.735000,0.000000,120.200000,5.000000,2.715000,761.000000,674.500000
max,5.706847,118.403456,37.500000,6.610000,343.000000,343.400000,5.000000,99.900000,9603.000000,899.000000


In [16]:
# 'Visualising Elephant Jasmin Movements'
# Creating and visualizing a scatter plot on Mapbox
fig = px.scatter_mapbox(df, lat='lat', lon='long',
                  animation_frame = 'GMTDate', animation_group = 'GMTTime',
                  color='Speed', size='Speed',
                  color_continuous_scale=px.colors.cyclical.IceFire,
                  size_max=50, zoom=1.75, hover_name='ID', 
                  hover_data = ['GMTDate', 'GMTTime', 'lat','long'], 
                  title = 'Visualising Elephant Jasmin Movements')

fig.update_layout(
        height=800,
        margin={'l': 0, 't': 0, 'b': 0, 'r': 0},
        mapbox={
            'center': {'lon': 118, 'lat': 5},
            'accesstoken':"pk.eyJ1IjoibmFlaW1hIiwiYSI6ImNsNDRoa295ZDAzMmkza21tdnJrNWRqNmwifQ.-cUTmhr1Q03qUXJfQoIKGQ",
            'style': 'satellite-streets',
            #'style': "open-street-map",
             #'style': 'outdoors',
            'center': {'lon': 118, 'lat': 5},
            'zoom': 8})
fig.show()

In [7]:
g = Graph()
ID = Namespace('DGFC_')
SOSA = Namespace('http://www.w3.org/ns/sosa/')
lat = Namespace('http://www.w3.org/2003/01/geo/wgs84_pos#')
long =Namespace('http://www.w3.org/2003/01/geo/wgs84_pos#')
alt = Namespace('http://www.w3.org/2003/01/geo/wgs84_pos#')
UNIT= Namespace('http://qudt.org/vocab/unit')
schema = Namespace('http://schema.org/')
uri=URIRef('http://www.w3.org/2000/01/rdf-schema#')
OBSPRO= Namespace('http://www.w3.org/ns/sosa/ObservableProperty/')
TIME = Namespace('http://www.w3.org/2006/time#')
VOID = Namespace('http://rdfs.org/ns/void#')
XMLNS = Namespace('http://www.w3.org/XML/1998/namespace')

In [8]:
for index, row in df.iterrows():
    g.add((URIRef(ID+row['ID']), RDF.type, SOSA.Observation))
    
    g.add((URIRef(ID+row['ID']), SOSA.Observation, Literal(row['ID'], datatype=XSD.string)))
    
    g.add((URIRef(ID+row['ID']), URIRef(schema+'DGFC/elephant#Jasmin'), Literal(row['ID'], datatype=XSD.string) ))  
    
    g.add((URIRef(ID+row['ID']), TIME.localDate, Literal(row['LocalDate'], datatype=XSD.date)))

    
    g.add((URIRef(ID+row['ID']), TIME.localTime, Literal(row['LocalTime'], datatype=XSD.time)))

    
    g.add((URIRef(ID+row['ID']), TIME.gMTDate, Literal(row['GMTDate'], datatype=XSD.date)))

    
    g.add((URIRef(ID+row['ID']), TIME.gMTTime, Literal(row['GMTTime'], datatype=XSD.time)))

  
    g.add((URIRef(ID+row['ID']), lat.lat, Literal(row['lat'], datatype=XSD.float)))

    g.add((URIRef(ID+row['ID']), long.long, Literal(row['long'], datatype=XSD.float)))
    
    g.add((URIRef(ID+row['ID']), OBSPRO.Temperature, Literal(row['Temperature'], datatype=XSD.double)))
   
    
    g.add((URIRef(ID+row['ID']), OBSPRO.Speed, Literal(row['Speed'], datatype=XSD.float)))
     
    g.add((URIRef(ID+row['ID']), alt.alt, Literal(row['Altitude'], datatype=XSD.float)))
   

    g.add((URIRef(ID+row['ID']), OBSPRO.Direction, Literal(row['Direction'], datatype=XSD.float)))
    g.add((URIRef(ID+row['ID']), OBSPRO.Distance, Literal(row['Distance'], datatype=XSD.float)))

    g.add((URIRef(ID+row['ID']), OBSPRO.HDOP, Literal(row['HDOP'], datatype=XSD.integer) ))
    g.add((URIRef(ID+row['ID']), OBSPRO.Cov, Literal(row['Cov'], datatype=XSD.integer) ))
    g.add((URIRef(ID+row['ID']), OBSPRO.Count, Literal(row['Count'], datatype=XSD.integer) ))

In [9]:
print(g.serialize(format='turtle')).head(10)

@prefix ns1: <http://www.w3.org/ns/sosa/ObservableProperty/> .
@prefix ns2: <http://www.w3.org/2003/01/geo/wgs84_pos#> .
@prefix ns3: <http://www.w3.org/2006/time#> .
@prefix ns4: <http://www.w3.org/ns/sosa/> .
@prefix ns5: <http://schema.org/DGFC/elephant#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<DGFC_100SAT32> a ns4:Observation ;
    ns5:Jasmin "100SAT32"^^xsd:string ;
    ns2:alt "NaN"^^xsd:float ;
    ns2:lat "5.676665"^^xsd:float ;
    ns2:long "118.39331299999999"^^xsd:float ;
    ns3:gMTDate "2011-11-13"^^xsd:date ;
    ns3:gMTTime "07:40:34"^^xsd:time ;
    ns3:localDate "2011-11-13"^^xsd:date ;
    ns3:localTime "03:40:34"^^xsd:time ;
    ns1:Count 100 ;
    ns1:Cov 4 ;
    ns1:Direction "0"^^xsd:float ;
    ns1:Distance "299"^^xsd:float ;
    ns1:HDOP 0.91 ;
    ns1:Speed "0.24"^^xsd:float ;
    ns1:Temperature 3.3e+01 ;
    ns4:Observation "100SAT32"^^xsd:string .

<DGFC_101SAT32> a ns4:Observation ;
    ns5:Jasmin "101SAT32"^^xsd:string ;
    ns2:alt "NaN"^^x

AttributeError: 'NoneType' object has no attribute 'head'

In [ ]:
# saving ontology to disk
g.serialize("SAT32Jasmin.rdf", format="ttl")

In [ ]:
# adding serialized data to stardog 

conn_details = {
  'endpoint': 'http://localhost:5820',
  'username': 'admin',
  'password': 'admin'
}
with stardog.Admin(**conn_details) as admin:
    Jasmin = admin.new_database('Jasmin')

conn = stardog.Connection('Jasmin', **conn_details)

conn.begin()

conn.add(
    stardog.content.File('Jasmin.rdf', stardog.content_types.TURTLE),
)

conn.commit()